# Ontological connection checking

Here I want to play around with ontological type checking in `pyiron_workflow` using `semantikon`'s `u` annotations.

In [4]:
from rdflib import Namespace, RDFS

import pyiron_workflow as pwf
from pyiron_workflow import knowledge
from semantikon.metadata import u
from semantikon import ontology  # get_knowledge_graph, validate_values
from pyiron_workflow.channels import ChannelConnectionError
from pyiron_workflow.nodes.composite import FailedChildError


EX = Namespace("http://www.example.org/")

class Meal: ...

class Garbage: ...

@pwf.as_function_node("pizza")
def prepare_pizza() -> u(Meal, uri=EX.Pizza):
    return Meal()

@pwf.as_function_node("unidentified_meal")
def prepare_non_ontological_meal() -> Meal:
    return Meal()

@pwf.as_function_node("rice")
def prepare_rice() -> u(Meal, uri=EX.Rice):
    return Meal()

@pwf.as_function_node("garbage")
def prepare_garbage() -> u(Garbage, uri=EX.Garbage):
    return Garbage()

@pwf.as_function_node("garbage")
def prepare_unhinted_garbage():
    return Garbage()

@pwf.as_function_node("verdict")
def eat(meal: u(Meal, uri=EX.Meal)) -> str:
    return f"Yummy {meal.__class__.__name__} meal"

@pwf.as_function_node("verdict")
def eat_pizza(meal: u(Meal, uri=EX.Pizza)) -> str:
    return f"Yummy {meal.__class__.__name__} pizza"

In [5]:
# Both fully hinted
wf = pwf.Workflow("ontoflow")
wf.make = prepare_pizza()
wf.eat = eat_pizza(wf.make)
wf()

{'eat__verdict': 'Yummy Meal pizza'}

In [6]:
ontology.validate_values(knowledge.parse_workflow(wf))

{'missing_triples': [], 'incompatible_connections': []}

In [7]:
# Upstream type hint is missing
wf = pwf.Workflow("no_type")
wf.make = prepare_unhinted_garbage()
wf.eat = eat_pizza(wf.make)
try:
    wf()
except FailedChildError as e:
    print(e)

/no_type encountered error in child: {'/no_type/eat.accumulate_and_run': TypeError("The channel /no_type/eat.meal cannot take the value `<__main__.Garbage object at 0x133a73410>` (<class '__main__.Garbage'>) because it is not compliant with the type hint typing.Annotated[__main__.Meal, ('uri', rdflib.term.URIRef('http://www.example.org/Pizza'))]")}


In [8]:
# Upstream type hint is wrong
wf = pwf.Workflow("no_type")
wf.make = prepare_garbage()
try:
    wf.eat = eat_pizza(wf.make)
except ChannelConnectionError as e:
    print(e)

The channel /no_type/make.garbage (<class 'pyiron_workflow.mixin.injection.OutputDataWithInjection'>) has the correct type (<class 'pyiron_workflow.channels.OutputData'>) to connect with /eat_pizza.meal (<class 'pyiron_workflow.channels.InputData'>), but is not a valid connection.Please check type hints, etc. /no_type/make.garbage.type_hint = typing.Annotated[__main__.Garbage, ('uri', rdflib.term.URIRef('http://www.example.org/Garbage'))]; /eat_pizza.meal.type_hint = typing.Annotated[__main__.Meal, ('uri', rdflib.term.URIRef('http://www.example.org/Pizza'))]


So far, so good: `u` decoration has no negative impact on the existing type hint checking procedures

In [9]:
# Upstream ontological hint is missing
wf = pwf.Workflow("no_ontology")
wf.make = prepare_non_ontological_meal()
wf.eat = eat_pizza(wf.make)
wf()
# This should stop working once I implement and activate ontological typing: ontological hint missing
# I guess that unlike a missing type hint, where we may get lucky at runtime and receive the right type,
# here we should fail at connection time

{'eat__verdict': 'Yummy Meal pizza'}

In [10]:
ontology.validate_values(knowledge.parse_workflow(wf))
# Or should it stop? `validate_values` doesn't seem to care...

{'missing_triples': [], 'incompatible_connections': []}

In [11]:
# Upstream ontological hint is WRONG
wf = pwf.Workflow("failed_ontology")
wf.make = prepare_rice()
# This should stop working once I implement and activate ontological typing: ontological hint mismatch
wf.eat = eat_pizza(wf.make)
wf()

{'eat__verdict': 'Yummy Meal pizza'}

In [12]:
ontology.validate_values(knowledge.parse_workflow(wf))

{'missing_triples': [],
 'incompatible_connections': [(rdflib.term.URIRef('failed_ontology.eat.inputs.meal'),
   rdflib.term.URIRef('failed_ontology.make.outputs.rice'),
   [rdflib.term.URIRef('http://www.example.org/Pizza')],
   [rdflib.term.URIRef('http://www.example.org/Rice')])]}

In [13]:
# Downstream ontological hint is less specific
wf = pwf.Workflow("relaxed_ontology")
wf.make = prepare_rice()
wf.eat = eat(wf.make)
wf()
# This should work fine

{'eat__verdict': 'Yummy Meal meal'}

In [14]:
ontology.validate_values(knowledge.parse_workflow(wf))

{'missing_triples': [],
 'incompatible_connections': [(rdflib.term.URIRef('relaxed_ontology.eat.inputs.meal'),
   rdflib.term.URIRef('relaxed_ontology.make.outputs.rice'),
   [rdflib.term.URIRef('http://www.example.org/Meal')],
   [rdflib.term.URIRef('http://www.example.org/Rice')])]}

This is not desirable, but is a known outcome from the semantikon notebook. The issue is that we need to inform the graph of the subclass relationship

In [15]:
graph = knowledge.parse_workflow(wf)
graph.add((EX.Rice, RDFS.subClassOf, EX.Meal))
ontology.validate_values(graph)

{'missing_triples': [], 'incompatible_connections': []}

In the pizza example we need to first get the knowledge graph _then_ tell it that `EX.Pizza` is a subclass of `EX.Meal`. I believe I will certainly need to be able to fulfill `get_workflow_dict` from inside `pyiron_workflow`, but I need to see how it will be possible to get these sort of subclass registrations working universally and not for some specific knowledge graph... To this end, the clothing example might actually be _easier_, because there is no extra-graph step...

# Ontological triples

Ok, that's not perfect but it's pretty damned good; let's try it with the clothes example

In [16]:
from rdflib import OWL, Namespace

import pyiron_workflow as pwf
from pyiron_workflow import knowledge
from semantikon.metadata import u
from semantikon import ontology

EX = Namespace("http://www.example.org/")

class Clothes:
    pass

@pwf.as_function_node
def wash(clothes: Clothes) -> u(
    Clothes,
    triples=(EX.hasProperty, EX.cleaned),
    derived_from="inputs.clothes"
):
    ...
    return clothes

@pwf.as_function_node
def dye(clothes: Clothes, color="blue") -> u(
    Clothes,
    triples=(EX.hasProperty, EX.color),
    derived_from="inputs.clothes",
):
    ...
    return clothes

@pwf.as_function_node
def sell(
    clothes: u(
        Clothes, restrictions=(
            ((OWL.onProperty, EX.hasProperty), (OWL.someValuesFrom, EX.cleaned)),
            ((OWL.onProperty, EX.hasProperty), (OWL.someValuesFrom, EX.color))
        )
    )
) -> int:
    return 10

In [17]:
my_correct_wf = pwf.Workflow("my_correct_workflow")
my_correct_wf.dyed_clothes = dye(Clothes())
my_correct_wf.washed_clothes = wash(my_correct_wf.dyed_clothes)
my_correct_wf.money = sell(my_correct_wf.washed_clothes)
my_correct_wf()

{'money__10': 10}

In [18]:
ontology.validate_values(knowledge.parse_workflow(my_correct_wf))

{'missing_triples': [], 'incompatible_connections': []}

In [19]:
@pwf.as_macro_node
def my_correct_macro(self, clothes: Clothes):
    self.dyed_clothes = dye(clothes)
    self.washed_clothes = wash(self.dyed_clothes)
    self.money = sell(self.washed_clothes)
    return self.money

correct_m = my_correct_macro(Clothes())
correct_m()

{'money': 10}

In [20]:
ontology.validate_values(knowledge.parse_workflow(correct_m))

{'missing_triples': [], 'incompatible_connections': []}

In [21]:
my_wrong_wf = pwf.Workflow("my_wrong_workflow")
my_wrong_wf.washed_clothes = wash(Clothes())
my_wrong_wf.money = sell(my_wrong_wf.washed_clothes)
my_wrong_wf()

{'money__10': 10}

In [22]:
ontology.validate_values(knowledge.parse_workflow(my_wrong_wf))

{'missing_triples': [(rdflib.term.URIRef('my_wrong_workflow.money.inputs.clothes'),
   rdflib.term.URIRef('http://www.example.org/hasProperty'),
   rdflib.term.URIRef('http://www.example.org/color'))],
 'incompatible_connections': []}

In [23]:
@pwf.as_macro_node
def my_wrong_macro(self, clothes: Clothes):
    self.washed_clothes = wash(clothes)
    self.money = sell(self.washed_clothes)
    return self.money

wrong_m = my_wrong_macro(Clothes())
wrong_m()

{'money': 10}

In [24]:
ontology.validate_values(knowledge.parse_workflow(wrong_m))

{'missing_triples': [(rdflib.term.URIRef('my_wrong_macro.money.inputs.clothes'),
   rdflib.term.URIRef('http://www.example.org/hasProperty'),
   rdflib.term.URIRef('http://www.example.org/color'))],
 'incompatible_connections': []}

In [25]:
@pwf.as_function_node
def dye_with_cancel(clothes: Clothes, color="blue") -> u(
    Clothes,
    triples=(EX.hasProperty, EX.color),
    derived_from="inputs.clothes",
    cancel=(EX.hasProperty, EX.cleaned)
):
    return clothes

In [26]:
my_wf_with_wrong_order = pwf.Workflow("my_workflow_with_wrong_order")
my_wf_with_wrong_order.washed_clothes = wash(Clothes())
my_wf_with_wrong_order.dyed_clothes = dye_with_cancel(my_wf_with_wrong_order.washed_clothes)
my_wf_with_wrong_order.money = sell(my_wf_with_wrong_order.dyed_clothes)
my_wf_with_wrong_order()

{'money__10': 10}

In [27]:
ontology.validate_values(knowledge.parse_workflow(my_wf_with_wrong_order))

{'missing_triples': [(rdflib.term.URIRef('my_workflow_with_wrong_order.money.inputs.clothes'),
   rdflib.term.URIRef('http://www.example.org/hasProperty'),
   rdflib.term.URIRef('http://www.example.org/cleaned'))],
 'incompatible_connections': []}

This example produces expected outcomes the whole way through.

As a first stage then, we can internally create type-validated connections, search for a graph root, transform the entire workflow graph to a "parsed" graph, feed that to semantikon, and reverse the connection iff we encounter a problem.
This is not the most computationally efficient approach, but should be pretty robust and very fast to implement.

# Going further?

Ok, what about if we make it a macro?
How does validation proceed making a connection _inside_ a subgraph?
If the workflow->graph is parsing subgraphs fine, and the graph->ontological validation is fine, then this might get inefficient but should work out-of-the-box.

# End of section